In [1]:
import spacy

spacy.cli.download("de_core_news_md")  # oder _lg für grosses Modell

2023-11-17 10:59:08.605924: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-17 10:59:08.605965: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-17 10:59:08.605991: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-17 10:59:08.613196: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-17 10:59:09.279672: W tensorflow/compiler/

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 MB 29.4 MB/s eta 0:00:00m eta 0:00:010:00:01
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_md')



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
nlp = spacy.load('de_core_news_md')

dok = nlp("Universität und Fachhochschule sind sich ähnlich. Studenten und Blumenkohl nicht.")

uni = dok[0]
fh = dok[2]

#zur Sicherheit
print(uni.has_vector)
print(fh.has_vector)

/home/yhutter/GitRepos/fhgr-ta/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True
True


In [3]:
uni.similarity(fh)

0.7625131607055664

In [4]:
stud = dok[7]
blum = dok[9]

print(C)
print(blum.has_vector)

stud.similarity(blum)

True
True


0.0755448266863823

In [5]:
import numpy as np

def most_similar(word, topn=5):
    word = nlp.vocab[str(word)]# Liste von Wörtern mit Ähnlichkeiten usw.
    queries = [
        w for w in word.vocab
        if w.prob >= -25 and np.count_nonzero(w.vector)
    ] # nur solche Wörter nehmen, die über einer gewissen Schwelle liegen und nicht nur Nullen enthalten
    by_similarity = sorted(queries, key=lambda w: word.similarity(w), reverse=True) #sortieren
    return [(w.lower_, w.similarity(word)) for w in by_similarity[:topn+1] if w.lower_ != word.lower_] #druckbar aufbereiten: Paar aus Wort und Ähnlichkeitsmass

for wort in ["Blumenkohl", "Student", "Fachhochschule"]:
    print(f"Am ähnlichsten zu {wort} sind: {most_similar(wort, topn=5)}")

Am ähnlichsten zu Blumenkohl sind: [('i.tr.', 0.5610805153846741), ('biol.', 0.2679746747016907), ('tägl.', 0.2484276294708252), ('tägl', 0.2484276294708252), ('ähnlich', 0.24624262750148773)]
Am ähnlichsten zu Student sind: [('studenten', 0.6420350670814514), ('univ', 0.5433400273323059), ('jur.', 0.5411397218704224), ('b.sc.', 0.5019005537033081), ('b.a.', 0.5019005537033081)]
Am ähnlichsten zu Fachhochschule sind: [('univ.', 0.7625131607055664), ('universität', 0.7625131607055664), ('abt.', 0.6067792177200317), ('b.sc.', 0.5304638147354126), ('b.a.', 0.5304638147354126)]


## Für ganze Sätze...

(Schlagzeilen "Blick" vom 06.10.2022)

In [6]:
import pandas

Satz1 = "Russland steht am Dnipro vor strategischem Dilemma"
Satz2 = "Borrell fordert Ende der Gewalt"
Satz3 = "Hier ergeben sich russische Soldaten in Cherson"
Satz4 = "Skandal-Brasserie Lorraine verweigert SVP den Service"
Satz5 = "Frustrierte Russen Soldaten schimpfen über ihre Versorgung"
Satz6 = "Gasindustrie wirbt für Gasspeicher"
Satz7 = "Anteil der Wärmepumpen wächst"
Satz8 = "Leverkusen-Stars verabschieden sich emotional von Seoane"
Satz9 = "Sevilla entlässt Trainer"
Satz10 = "Autofahrer fährt Blinden an und flüchtet"
Satz11 = "Aktivisten blockieren Autobahn"
Satz12 = "Leverkusen entlässt Trainer Seoane"
Satz13 = "Experte Mark Iten macht den Energie-Check"
Satz14 = "Wanderin bleibt im Entlebuch im Schlamm stecken"

nlp1 = nlp(Satz1)
nlp2 = nlp(Satz2)
nlp3 = nlp(Satz3)
nlp4 = nlp(Satz4)
nlp5 = nlp(Satz5)
nlp6 = nlp(Satz6)
nlp7 = nlp(Satz7)
nlp8 = nlp(Satz8)
nlp9 = nlp(Satz9)
nlp10 = nlp(Satz10)
nlp11 = nlp(Satz11)
nlp12 = nlp(Satz12)
nlp13 = nlp(Satz13)
nlp14 = nlp(Satz14)

saetze = [Satz1, Satz2, Satz3, Satz4, Satz5, Satz6, Satz7, Satz8, Satz9, Satz10, Satz11, Satz12, Satz13, Satz14]
nlps = [nlp1, nlp2, nlp3, nlp4, nlp5, nlp6, nlp7, nlp8, nlp9, nlp10, nlp11, nlp12, nlp13, nlp14]

aehnlich = []
#Achtung, doppelte Arbeit!
for satznlp1 in nlps:
    aehnlich_zeile = []
    for satznlp2 in nlps:
        aehnlich_zeile.append(satznlp1.similarity(satznlp2))
    aehnlich.append(aehnlich_zeile)

aehnlich_df = pandas.DataFrame(aehnlich)
aehnlich_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1.000000,0.395553,0.312715,0.289470,0.271375,0.099344,0.239892,0.258239,0.247062,0.366869,0.322885,0.229572,0.258874,0.322950
1,0.395553,1.000000,0.468611,0.289724,0.522609,0.221215,0.585938,0.409845,0.247565,0.384669,0.397442,0.233980,0.273177,0.175357
2,0.312715,0.468611,1.000000,0.206396,0.640072,0.205099,0.404833,0.571491,0.268880,0.539788,0.452391,0.236551,0.294569,0.238216
3,0.289470,0.289724,0.206396,1.000000,0.310510,0.161174,0.229435,0.357084,0.177425,0.260760,0.263719,0.176408,0.551274,0.170099
4,0.271375,0.522609,0.640072,0.310510,1.000000,0.265342,0.416491,0.533535,0.241342,0.605127,0.568671,0.220040,0.349421,0.110519
5,0.099344,0.221215,0.205099,0.161174,0.265342,1.000000,0.403946,0.117091,0.111605,0.202589,0.243845,0.057647,0.274202,0.133035
6,0.239892,0.585938,0.404833,0.229435,0.416491,0.403946,1.000000,0.329751,0.170448,0.407615,0.296125,0.134321,0.292485,0.233866
7,0.258239,0.409845,0.571491,0.357084,0.533535,0.117091,0.329751,1.000000,0.383608,0.468215,0.299122,0.470761,0.419675,0.093876
8,0.247062,0.247565,0.268880,0.177425,0.241342,0.111605,0.170448,0.383608,1.000000,0.307867,0.190154,0.882280,0.399968,0.109937
9,0.366869,0.384669,0.539788,0.260760,0.605127,0.202589,0.407615,0.468215,0.307867,1.000000,0.533563,0.245251,0.278854,0.155141


In [7]:
#hohe Übereinstimmungen suchen - also grosse Werte unter 1 ...

#Vektor mit Max-Wert pro Zeile (aber < 1)
max_zeile = aehnlich_df[aehnlich_df < 1].max()
max_zeile

0     0.395553
1     0.585938
2     0.640072
3     0.551274
4     0.640072
5     0.403946
6     0.585938
7     0.571491
8     0.882280
9     0.605127
10    0.568671
11    0.882280
12    0.551274
13    0.322950
dtype: float64

In [8]:
#am ähnlichsten

print(Satz12)
print(Satz9)

Leverkusen entlässt Trainer Seoane
Sevilla entlässt Trainer


In [9]:
print(Satz3)
print(Satz5)

Hier ergeben sich russische Soldaten in Cherson
Frustrierte Russen Soldaten schimpfen über ihre Versorgung
